In [3]:
# 파이썬 2와 파이썬 3 지원
from __future__ import division, print_function, unicode_literals

# 공통
import numpy as np
import os

# 일관된 출력을 위해 유사난수 초기화
def reset_graph(seed=42):
    tf.compat.v1.reset_default_graph()
    tf.compat.v1.set_random_seed(seed)
    np.random.seed(seed)

# 맷플롯립 설정
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# 한글출력
plt.rcParams['font.family'] = 'NanumBarunGothic'
plt.rcParams['axes.unicode_minus'] = False

# 그림을 저장할 폴더
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "rnn"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [20]:
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:

input_data = [1,2,3,4,5]

x = tf.placeholder(dtype=tf.float32)

y = x * 2


sess = tf.Session()

result = sess.run(y,feed_dict={x:input_data})


print(result)

[ 2.  4.  6.  8. 10.]


In [2]:
reset_graph()

n_inputs = 3
n_neurons = 5

#훈련을 하는 동안 텐서플로에 훈련 데이터를 전달하기 위해 사용한다.
X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])
hi = tf.random_normal(shape=[n_inputs, n_neurons])
print(type(hi))
                 
Wx = tf.constant(hi,dtype=tf.float32)
# Wy = tf.constant(tf.random_normal(shape=[n_neurons,n_neurons],dtype=tf.float32),name="Wy")
# b = tf.constant(tf.zeros([1, n_neurons], dtype=tf.float32))

# Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
# Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)

# print(Y0)
# init = tf.global_variables_initializer()

NameError: ignored

In [ ]:

print(Y1_val)

[[-1.         -0.90785277 -1.         -1.          1.        ]
 [ 0.46041214 -0.9532765  -0.9636991   0.9714266   0.16984603]
 [-0.9976139  -0.9438259  -1.         -0.9999977   1.        ]
 [-0.97473055 -0.9506037  -0.9999533  -0.9910697   0.99739045]]


In [21]:
reset_graph()

n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10

learning_rate = 0.001

X = tf.compat.v1.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.compat.v1.placeholder(tf.int32, [None])

basic_cell = tf.compat.v1.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

logits = tf.layers.dense(states, n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                          logits=logits)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/usr/local/lib/python3.7/dist-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:415: UserWarning: `tf.nn.rnn_cell.BasicRNNCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.SimpleRNNCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.BasicRNNCell` is deprecated and will be "
/usr/local/lib/python3.7/dist-packages/keras/engine/base_layer_v1.py:1684: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:236: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/usr/local/lib/python3.7/dist-packages/keras/engine/base_layer_v1.py:1676: UserWarning: `layer.apply` is 

In [26]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]
X_test = X_test.reshape((-1, n_steps, n_inputs))
X_valid = X_valid.reshape((-1, n_steps, n_inputs))

print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(60000, 28, 28)
(10000, 28, 28)
(60000,)
(10000,)
(55000, 784)
(5000, 28, 28)
(55000,)
(5000,)


In [23]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [24]:
n_epochs = 100
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "배치 데이터 정확도:", acc_batch, "검증 세트 정확도:", acc_valid)

0 배치 데이터 정확도: 0.91333336 검증 세트 정확도: 0.9266
1 배치 데이터 정확도: 0.93333334 검증 세트 정확도: 0.9474
2 배치 데이터 정확도: 0.93333334 검증 세트 정확도: 0.9516
3 배치 데이터 정확도: 0.96 검증 세트 정확도: 0.9488
4 배치 데이터 정확도: 0.9866667 검증 세트 정확도: 0.965
5 배치 데이터 정확도: 0.94666666 검증 세트 정확도: 0.963
6 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9678
7 배치 데이터 정확도: 0.97333336 검증 세트 정확도: 0.968
8 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9732
9 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9666
10 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9708
11 배치 데이터 정확도: 0.97333336 검증 세트 정확도: 0.9724
12 배치 데이터 정확도: 0.9533333 검증 세트 정확도: 0.9678
13 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9744
14 배치 데이터 정확도: 0.9866667 검증 세트 정확도: 0.9758
15 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9724
16 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9742
17 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.966
18 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9778
19 배치 데이터 정확도: 0.96666664 검증 세트 정확도: 0.9756
20 배치 데이터 정확도: 0.9866667 검증 세트 정확도: 0.976
21 배치 데이터 정확도: 0.9866667 검증 세트 정확도: 0.9788
22 배치 데이터 정확도: 0.97333336 검증 세트 정확도: 0.981
23 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.974
24 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.